In [135]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from IPython.display import display


In [109]:
df = pd.read_csv('./test-data.csv')
df['logs'] = df['logs'].map(lambda x: json.loads(x) if not pd.isnull(x) else pd.NA)


In [113]:
df.loc[df['participant_id'].isna(),'participant_id'] = 'test'

In [114]:
df

,participant_id,tasks,__INDEX__,participant_number,task,content,questions,logs,menu,items,command,question,object,timeLimit,filename,experimenter
0,blaine,"[""ProgressBar"",""DevTools""]",[5],0,ConsentForm,# Research Consent Form\n\n**Researcher**: Tov...,"[{""label"":""I understand and consent to the abo...","[{'type': 'START', 'timestamp': 1653548305434}...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,blaine,"[""ProgressBar"",""DevTools""]",[5],0,InformationScreen,\nIn order to complete this experiment you wil...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,blaine,"[""ProgressBar"",""DevTools""]",[5],0,Tutorial,NaN,NaN,<NA>,MarkingMenu,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",NaN,NaN,NaN,NaN,NaN,NaN
3,blaine,"[""ProgressBar"",""DevTools""]",[5],0,CommandSelection,NaN,NaN,<NA>,MarkingMenu,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",Clear formatting,NaN,NaN,NaN,NaN,NaN
4,blaine,"[""ProgressBar"",""DevTools""]",[5],0,CommandSelection,NaN,NaN,<NA>,MarkingMenu,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",Bold,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,test,"[""ProgressBar"",""DevTools""]",[5],1,DivergentTest,NaN,NaN,"[{'type': 'START', 'timestamp': 1674845372904}...",KeyboardShortcutsWithCheatsheet,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",NaN,Write down all of the original and creative\n ...,knife,30000.0,NaN,NaN
187,test,"[""ProgressBar"",""DevTools""]",[5],1,Questionnaire,NaN,"[""I felt creative while I was selecting comman...","[{'type': 'START', 'timestamp': 1674845407897}...",KeyboardShortcutsWithCheatsheet,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",NaN,NaN,NaN,NaN,NaN,NaN
188,test,"[""ProgressBar"",""DevTools""]",[5],1,NasaTlx,NaN,NaN,"[{'type': 'START', 'timestamp': 1674845412928}...",KeyboardShortcutsWithCheatsheet,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",NaN,NaN,NaN,NaN,NaN,NaN
189,test,"[""ProgressBar"",""DevTools""]",[5],1,S3Upload,NaN,NaN,"[{'type': 'START', 'timestamp': 1674845421983}]",NaN,NaN,NaN,NaN,NaN,NaN,participant_id,blaine@dgp.toronto.edu


# Datafram Creation

## Selection

In [115]:
selection = df[df['task'] == 'CommandSelection']

## Creativity

In [150]:
unusual_uses = df[df['task'] == 'DivergentTest']

# TODO: so the issue here is that we have multiple rows per group.

def extract_unusual_uses(g):
    for index, row in g.iterrows():
        if not isinstance(row['logs'], list):
            return pd.Series(pd.NA)

        for log in row['logs']:
            if log['type'] == 'DivergentTest':
                new_df = pd.DataFrame(log['values'], columns=['values'])
                new_df['topTwo'] = False
                new_df.loc[list(map(int,log['topTwo'])), 'topTwo'] = True
                return new_df
        
unusual_uses = unusual_uses.groupby(['participant_id', 'menu']).apply(extract_unusual_uses).reset_index()
del unusual_uses['level_2']
del unusual_uses[0]

In [151]:
unusual_uses

,participant_id,menu,values,topTwo
0,blaine,KeyboardShortcutsWithCheatsheet,NaN,NaN
1,blaine,MarkingMenu,NaN,NaN
2,blaine,ToolPalette,NaN,NaN
3,test,KeyboardShortcutsWithCheatsheet,as a hammer with the handle,True
4,test,KeyboardShortcutsWithCheatsheet,knife is hard,True
5,test,KeyboardShortcutsWithCheatsheet,huh. weird,False
6,test,MarkingMenu,hitting people over the head,False
7,test,MarkingMenu,clapping if you have two of them,False
8,test,MarkingMenu,pillow,True
9,test,MarkingMenu,drawing,True


In [118]:
tlx = df[df['task'] == 'NasaTlx']


def extract_tlx(row):
    if not isinstance(row['logs'], list):
        return pd.Series(pd.NA)
    
    for log in row['logs']:
        if log['type'] == 'NasaTlx':
            return pd.Series(log['responses'])
        
tlx = pd.concat([tlx, tlx.apply(extract_tlx, axis=1)], axis=1)
tlx

,participant_id,tasks,__INDEX__,participant_number,task,content,questions,logs,menu,items,...,timeLimit,filename,experimenter,0,Effort,Frustration,Mental Demand,Performance,Physical Demand,Temporal Demand
28,blaine,"[""ProgressBar"",""DevTools""]",[5],0,NasaTlx,NaN,NaN,<NA>,MarkingMenu,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",...,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
55,blaine,"[""ProgressBar"",""DevTools""]",[5],0,NasaTlx,NaN,NaN,<NA>,ToolPalette,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",...,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
82,blaine,"[""ProgressBar"",""DevTools""]",[5],0,NasaTlx,NaN,NaN,"[{'type': 'START', 'timestamp': 1653548343304}...",KeyboardShortcutsWithCheatsheet,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
120,test,"[""ProgressBar"",""DevTools""]",[5],1,NasaTlx,NaN,NaN,"[{'type': 'START', 'timestamp': 1674845180245}...",MarkingMenu,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",...,NaN,NaN,NaN,NaN,12.0,9.0,12.0,9.0,8.0,16.0
154,test,"[""ProgressBar"",""DevTools""]",[5],1,NasaTlx,NaN,NaN,"[{'type': 'START', 'timestamp': 1674845315353}...",ToolPalette,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",...,NaN,NaN,NaN,NaN,14.0,10.0,7.0,10.0,8.0,15.0
188,test,"[""ProgressBar"",""DevTools""]",[5],1,NasaTlx,NaN,NaN,"[{'type': 'START', 'timestamp': 1674845412928}...",KeyboardShortcutsWithCheatsheet,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",...,NaN,NaN,NaN,NaN,16.0,12.0,10.0,12.0,14.0,14.0


In [124]:
questionnaire = df[df['task'] == 'Questionnaire']

def extract_questionnaire(row):
    if not isinstance(row['logs'], list):
        return pd.Series(pd.NA)

    for log in row['logs']:
        if log['type'] == 'QuestionnaireResponses':
            return pd.Series(log['responses'])
        
questionnaire = pd.concat([questionnaire, questionnaire.apply(extract_questionnaire, axis=1)], axis=1)

In [125]:
questionnaire

,participant_id,tasks,__INDEX__,participant_number,task,content,questions,logs,menu,items,command,question,object,timeLimit,filename,experimenter,0,I felt creative while I was selecting commands.,I would like to use that command selection technique while doing a creative task like drawing or writing a story.
27,blaine,"[""ProgressBar"",""DevTools""]",[5],0,Questionnaire,NaN,"[""I felt creative while I was selecting comman...",<NA>,MarkingMenu,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
54,blaine,"[""ProgressBar"",""DevTools""]",[5],0,Questionnaire,NaN,"[""I felt creative while I was selecting comman...",<NA>,ToolPalette,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
81,blaine,"[""ProgressBar"",""DevTools""]",[5],0,Questionnaire,NaN,"[""I felt creative while I was selecting comman...","[{'type': 'START', 'timestamp': 1653548342186}...",KeyboardShortcutsWithCheatsheet,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
119,test,"[""ProgressBar"",""DevTools""]",[5],1,Questionnaire,NaN,"[""I felt creative while I was selecting comman...","[{'type': 'START', 'timestamp': 1674845174945}...",MarkingMenu,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,51.0
153,test,"[""ProgressBar"",""DevTools""]",[5],1,Questionnaire,NaN,"[""I felt creative while I was selecting comman...","[{'type': 'START', 'timestamp': 1674845310603}...",ToolPalette,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,65.0
187,test,"[""ProgressBar"",""DevTools""]",[5],1,Questionnaire,NaN,"[""I felt creative while I was selecting comman...","[{'type': 'START', 'timestamp': 1674845407897}...",KeyboardShortcutsWithCheatsheet,"[{""icon"":{},""label"":""Undo"",""angle"":3.141592653...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,74.0
